In [ ]:
! pip install transformers
! pip install datasets
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import f1_score


In [2]:

from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time
# from torchmetrics import MeanSquaredError
import torch
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [3]:

import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation

Mounted at /content/drive
/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation


In [5]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
df2 = pd.read_json('/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation/ERAI_Dataset_pairwise_test_with_translation.json')
df = df.dropna(axis = 0)

pd.set_option('display.max_rows', None)

df.reset_index(drop = True,inplace = True)

df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
df['post2']=df['post2'].apply(lambda x:x.replace('\n',''))


In [6]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"
# 获取预测模型所使用的tokenizer
# AutoTokenizer, AutoModel,

#"bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [11]:

def generate_input_data(post,mpp_all_value,ml_all_value,max_len):

 
  
  all_value = pd.concat([mpp_all_value,ml_all_value],axis=1)
  
  
  full_text = []
  attention_mask = []
  # token_type_ids = []
  for i in post:
    if len(i)>max_len:
      i = i[:max_len]
    
    i = tokenizer.tokenize(i)
    
    text = ['[CLS]']+i+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    
  all_pad_tokens = pad_sequences(full_text, maxlen=max_len, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len, padding='post', truncating='post')
  
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),     
      'all_value': torch.tensor(all_value.values, dtype=torch.float),
     
  }

    

In [12]:
 class BertNerClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained("bert-base-chinese")
    self.drop = nn.Dropout(p=0.1)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask

    )[1]
    
    output = self.drop(output)
    
    
    return self.out(output)

In [20]:

class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      
      self.all_value = data['all_value']
     

    def __len__(self):

        return len(self.all_value)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.all_value[index]

In [21]:
def generate_dataloader(post,mpp_all_value,ml_all_value,max_len,batch_size):
  data = generate_input_data(post,mpp_all_value,ml_all_value,max_len)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  return dataloader


In [22]:
EPOCHS = 15


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_fn = nn.MSELoss().to(device)


label_num = 1
model= BertNerClassifier(label_num).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
def test_prediction(model, 
  test_dataloader, 
  loss_fn, 
  optimizer,
  device
  ):
   
  test_pred = []

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      
      targets = data[2].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )

      
      test_pred.append(outputs)
  return test_pred


In [30]:

def train_and_eval(model, 
  train_loader, 
  loss_fn, 
  optimizer,  
  device,
  epoch,
  label_num,
  test_dataloader1,
  test_dataloader2
  ):
  model.train()
  accuracy_state = 0
  
  for epo in range(epoch):
    
    
    train_loss = []
    
    train_predictions = []
    
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      
      targets = data[2].to(device)
      

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )
      

      loss = loss_fn(outputs, targets)
      
      

      train_loss.append(loss.item())
      
  
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    test_pred1 = test_prediction(model, 
      test_dataloader1, 
      loss_fn, 
      optimizer,
      device
      )

    test_pred2 = test_prediction(model, 
      test_dataloader2, 
      loss_fn, 
      optimizer,
      device
    )

    mpp_pred_label = []
    ml_pred_label = []

    for i, j in zip (test_pred1,test_pred2):
      if i[0][0]>j[0][0]:
        mpp_pred_label.append(1)
      else:
        mpp_pred_label.append(0)
      if i[0][1]>j[0][1]:
        ml_pred_label.append(0)
      else:
        ml_pred_label.append(1)
    
   
    
    final_train_loss = np.mean(train_loss)

    
    mpp_accuracy = accuracy_score(test_mpp_label, mpp_pred_label)
    ml_accuracy = accuracy_score(test_ml_label, ml_pred_label)

    if (mpp_accuracy+ml_accuracy)/2 > accuracy_state:
      accuracy_state = (mpp_accuracy+ml_accuracy)/2
      torch.save(model.state_dict(), 'double_value_regression.bin')



    print(
'Epoch [{}] Train Totol Loss: {:.4f}, TEST MPP ACCURACY: {:.4f}, Test Totol Loss: {:.4f}'
    .format(epo+1,final_train_loss,mpp_accuracy,ml_accuracy))

In [ ]:
random_seed=[171,354,550,667,985]

for i in random_seed:
  df = shuffle(df,random_state=i)

  train = df[0:156]
  # validation = df[137:156]
  test = df[156:200]

  # test = df[176:200]
  train_post1 = train['post1']
  train_post2 = train['post2']
  train_mmp1 = train['MPP1']
  train_mpp2 = train['MPP2']
  train_ml1 = train['ML1']
  train_ml2 = train['ML2']

  all_train_post = pd.concat([train_post1,train_post2],axis=0)
  all_train_mpp = pd.concat([train_mmp1,train_mpp2],axis=0)
  all_train_ml = pd.concat([train_ml1,train_ml2],axis=0)

  max_len = 512
  batch_size = 1
  train_dataloader = generate_dataloader(all_train_post,all_train_mpp,all_train_ml,max_len,batch_size)

  test_post1 = test['post1']
  test_post2 = test['post2']
  test_mmp1 = test['MPP1']
  test_mpp2 = test['MPP2']
  test_ml1 = test['ML1']
  test_ml2 = test['ML2']
  test_mpp_label = test['MPP_label']
  test_ml_label = test['ML_label']

  test_dataloader1 = generate_dataloader(test_post1,test_mmp1,test_ml1,max_len,1)
  test_dataloader2 = generate_dataloader(test_post2,test_mpp2,test_ml2,max_len,1)

  # label_num = 2
  # model= BertNerClassifier(label_num).to(device)
  label_num = 2
  model= BertNerClassifier(label_num).to(device)

  optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
  train_and_eval(model, 
    train_dataloader, 
    loss_fn, 
    optimizer,  
    device,
    EPOCHS,
    label_num,
    test_dataloader1,
    test_dataloader2
  )
  print('========================================================================================')
  print('========================================================================================')

In [ ]:
model.load_state_dict(torch.load('double_value_regression.bin', map_location=torch.device("cuda:0")))

<All keys matched successfully>

In [31]:

def generate_test_data(post,max_len):
 
  
  full_text = []
  attention_mask = []
  
  for i in post:
    if len(i)>max_len:
      i = i[:max_len]
    
    i = tokenizer.tokenize(i)
    
    text = ['[CLS]']+i+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    
  all_pad_tokens = pad_sequences(full_text, maxlen=max_len, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len, padding='post', truncating='post')
  
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),

  }


In [32]:

class testDataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index]

In [33]:
def generate_test_dataloader(post,max_len,batch_size):
  data = generate_test_data(post,max_len)
  data = testDataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  return dataloader

In [34]:
test_post1 = df2['post1']
test_post2 = df2['post2']

max_len = 512
test_dataloader1 = generate_test_dataloader(test_post1,max_len,1)
test_dataloader2 = generate_test_dataloader(test_post2,max_len,1)

In [37]:
def real_prediction(model, 
  test_dataloader, 
  loss_fn, 
  optimizer,
  device
  ):
   
  test_pred = []

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        
      )

      
      test_pred.append(outputs)
  return test_pred

In [39]:
test_pred1 = real_prediction(model, 
      test_dataloader1, 
      loss_fn, 
      optimizer,
      device
      )

test_pred2 = real_prediction(model, 
      test_dataloader2, 
      loss_fn, 
      optimizer,
      device
    )

mpp_pred_label = []
ml_pred_label = []

for i, j in zip (test_pred1,test_pred2):
  if i[0][0]>j[0][0]:
    mpp_pred_label.append(1)
  else:
    mpp_pred_label.append(0)
  if i[0][1]>j[0][1]:
    ml_pred_label.append(0)
  else:
    ml_pred_label.append(1)
    



In [40]:
df2['MPP_prediction'] = mpp_pred_label
df2['ML_prediction'] = ml_pred_label

In [41]:
df2.to_json('ERAI_pairwise_[aiml]_[1].json')

In [ ]:
df2